In [82]:
'''
0 0 0
0 21600000000000 23400000000000
1476000000000 2160000000000 3723000000000

Case #1: 0 0 0 0
Case #2: 6 30 0 0
Case #3: 1 2 3 0


5400000000000 5400000000000 5400000000000
10800000000000 32400000000000 34200000000000
23076000000000 23760000000000 25323000000000

Case #1: 0 0 0 0
Case #2: 0 30 0 0
Case #3: 1 2 3 0

0 11 719

Case #1: 0 0 0 1
'''
pass

In [115]:
a,b,c,d = 1,2,3,0

In [116]:
# a,b,c,d = 0,0,0,1

In [117]:
nsecs = a*60*60*10**9 + b*60*10**9 + c*10**9 + d

In [118]:
# nsecs = 39477000000000

In [119]:
nsecs

3723000000000

In [120]:
M = (60*720*10**9) 

In [121]:
nsecs, (720//60*nsecs)%(60*720*10**9), (720*nsecs)%(60*720*10**9) 

(3723000000000, 1476000000000, 2160000000000)

In [122]:
p = (720*nsecs)//M

In [123]:
p

62

In [92]:
nsecs, 12*nsecs - (p//60)*M, 720*nsecs - p*M

(3723000000000, 1476000000000, 2160000000000)

In [94]:
a,b,c = 3723000000000, 1476000000000, 2160000000000
d,e,f = abs(a-b), abs(b-c), abs(a-c)

In [96]:
d,e,f

(2247000000000, 684000000000, 1563000000000)

In [58]:
720/60

12.0

In [33]:
720/60

12.0

In [128]:
print(" ".join(["20"]*20))

20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20
